## Further Reaching Analysis: Linear Modeling

For our further reaching analysis, we want to check different predictors and understand how each of them influences the outcome variable Voltage. We discussed about whether to use RewP or voltage (mean/max) as the outcome variable and decided to use voltage. Our rationale is that when we were to use RewP, most of a selected predictors can not be included anymore as RewP is constructed by the a 

In [27]:
# HYPTERPARAMETERS
VOLTAGE_TO_ANALYZE = "mean_voltage"
SELECTED_CHANNEL="FCz"
FILT_DS_PATH="Dataset/ds004147-filtered"
RAW_DS_PATH="Dataset/ds004147"
OUTPUT_DIR=None  # None means don't save as csv
SUBJECTS_TO_INCLUDE=None  # None means all


In [28]:
# import all needed libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import statsmodels.api as sm
from statsmodels.formula.api import ols
from scipy.stats import pearsonr
from scipy.stats import chi2_contingency
import os
import mne
import numpy as np
import re
import pandas as pd
from tqdm import tqdm


#####################################################  This section is for creating the dummy data... Skip this when we have the real data  ############################################################################################ 

In [29]:
# Dummy Data Generation Function
def generate_dummy_values_csv(csv_name, num_values=100, median=0, spread=1, win_ratio=0.5):
    rewp_values = np.random.randn(num_values) * spread + median
    # Generate win/loss labels directly 
    prev_outcome = np.random.choice(['loss', 'win'], size=num_values, p=[1-win_ratio, win_ratio])
    performance = np.random.uniform(0.5, 1, size=num_values)
    outcome = np.random.choice(['loss', 'win'], size=num_values, p=[1-win_ratio, win_ratio])
    df = pd.DataFrame({"outcome": outcome, 'prev_outcome': prev_outcome, "performance": performance, 'RewP': rewp_values})
    df.to_csv(csv_name, index=False)
  
# Generation of Dummy Data as csv
generate_dummy_values_csv("data/lowlow_rewp_dummy_values.csv", num_values=100, median=7, spread=3, win_ratio=0.5)
generate_dummy_values_csv("data/midlow_rewp_dummy_values.csv", num_values=50, median=7.5, spread=3, win_ratio=0.5)
generate_dummy_values_csv("data/midhigh_rewp_dummy_values.csv", num_values=50, median=8, spread=3, win_ratio=0.7)
generate_dummy_values_csv("data/highhigh_rewp_dummy_values.csv", num_values=100, median=5.5, spread=3, win_ratio=0.7)

# Reading the generated Dummy Data
lowlow_df = pd.read_csv('data/lowlow_rewp_dummy_values.csv')
midlow_df = pd.read_csv('data/midlow_rewp_dummy_values.csv')
midhigh_df = pd.read_csv('data/midhigh_rewp_dummy_values.csv')
highhigh_df = pd.read_csv('data/highhigh_rewp_dummy_values.csv')

# Creating two new columns Task and Cue and assign the corresponding value to the Dummy Data
lowlow_df["task"] = ["low" for _ in range(len(lowlow_df))]
lowlow_df["cue"] = ["low" for _ in range(len(lowlow_df))]
midlow_df["task"] = ["mid" for _ in range(len(midlow_df))]
midlow_df["cue"] = ["low" for _ in range(len(midlow_df))]
midhigh_df["task"] = ["mid" for _ in range(len(midlow_df))]
midhigh_df["cue"] = ["high" for _ in range(len(midlow_df))]
highhigh_df["task"] = ["high" for _ in range(len(highhigh_df))]
highhigh_df["cue"] = ["high" for _ in range(len(highhigh_df))]

# Combining Dummy Data
df_list = [lowlow_df, midlow_df, midhigh_df, highhigh_df]
data = pd.concat(df_list, ignore_index=True)
data.to_csv("data/dummy_data.csv", index=False)

####################################################################################################################################################################################################################

In [30]:
CONDITION_BEH_MAPPING = {
    6: (1, 50, 1),
    16: (2, 50, 1),
    26: (2, 80, 1),
    36: (3, 80, 1),
    7: (1, 50, 0),
    17: (2, 50, 0),
    27: (2, 80, 0),
    37: (3, 80, 0),
}

CONDITION_MAPPING = {
    'Win LL': 6,
    'Loss LL': 7,
    'Win ML': 16,
    'Loss ML': 17,
    'Win MH': 26,
    'Loss MH': 27,
    'Win HH': 36,
    'Loss HH': 37,
}

# import all needed helper functions
def create_df_from_beh_tsv(tsv_name):
    df = pd.read_csv(tsv_name, sep='\t')
    df = df[['block', 'trial', 'task', 'prob', 'outcome', 'optimal', 'rt']]
    df['id'] = df['block'] * df['trial']
    # Calculate Z-score of RT within each task
    task_mean_rt = df.groupby('task')['rt'].transform('mean')
    task_std_rt = df.groupby('task')['rt'].transform('std')
    df['rt_z_score'] = (df['rt'] - task_mean_rt) / task_std_rt
    # Calculate percentile of RT within each task
    task_percentiles = df.groupby('task')['rt'].transform(lambda x: (x.rank() - 1) / len(x) * 100)
    df['rt_percentile'] = task_percentiles
    # Calculate residuals
    residuals = df.groupby('task')['rt'].transform(lambda x: x - x.mean())
    df['residuals'] = residuals
    return df[['id', 'task', 'prob', 'outcome', 'optimal', 'rt_z_score', 'rt_percentile', 'residuals']]

def create_outcome_figure(df1, df2, title, ax):
    ax.boxplot([df1[VOLTAGE_TO_ANALYZE], df2[VOLTAGE_TO_ANALYZE]])
    ax.set_xticks([1, 2])
    ax.set_xticklabels(['Loss (prev)', 'Win (prev)'])
    ax.set_ylabel(VOLTAGE_TO_ANALYZE)
    ax.set_title(title)
    
def analyze_categorical_feature(feature, data, second_feature=None):
    if second_feature:
        # Pair up the first and second features into one
        data[f'{feature}-{second_feature}'] = data[feature] + '-' + data[second_feature]
        feature = f'{feature}-{second_feature}'

    # Grouping data by the feature and extracting 'RewP' values for each group
    feature_groups = data.groupby(feature)[VOLTAGE_TO_ANALYZE].apply(list)
    
    # Plotting boxplot
    plt.figure(figsize=(8, 6))
    plt.boxplot(feature_groups.values, labels=feature_groups.index)
    plt.xlabel(feature)
    plt.ylabel(VOLTAGE_TO_ANALYZE)
    plt.title(f'Boxplot of {VOLTAGE_TO_ANALYZE} by {feature}')
    plt.xticks(rotation=90)
    plt.tight_layout() 
    plt.show()
    
    # Fit ANOVA model
    model = ols(f'{VOLTAGE_TO_ANALYZE} ~ ' + feature, data=data).fit()
    anova_table = sm.stats.anova_lm(model, typ=2)
    print(anova_table)


    
def analyze_numerical_feature(feature, data):
    # Scatter plot
    plt.figure(figsize=(8, 6))
    plt.scatter(data[feature], data[VOLTAGE_TO_ANALYZE], alpha=0.5)
    plt.xlabel(feature)
    plt.ylabel(VOLTAGE_TO_ANALYZE)
    plt.title(f'Scatter Plot of {VOLTAGE_TO_ANALYZE} vs {feature}')
    plt.grid(True)
    plt.show()
    # Correlation analysis
    correlation, p_value = pearsonr(data[feature], data[VOLTAGE_TO_ANALYZE])
    print(f"Correlation between {feature} and {VOLTAGE_TO_ANALYZE}: {correlation}")
    print(f"P-value: {p_value}")
    
def convert_condition_id_to_beh_names(condition_id):
    return CONDITION_BEH_MAPPING[condition_id]


def filter_df_by_kwargs(df, **kwargs):
    mask = pd.Series([True] * len(df))
    for col, value in kwargs.items():
        mask = mask & (df[col] == value)
    return df[mask]


def extract_number(value):
    try:
        import re
        return float(re.findall(r'\d+', value)[0])
    except (IndexError, ValueError):
        return np.nan
    

def retrieve_data(selected_channel, filt_ds_path, subjects_to_include, raw_ds_path, output_dir):
    df = pd.DataFrame(columns=["max_voltage", "mean_voltage", "subject", "outcome", "task", "cue", "prev_outcome"])
    max_voltages = []
    mean_voltages = []
    subjects = []
    outcomes = []
    tasks = []
    cues = []
    prev_outcomes = []
    subject_names = [d for d in os.listdir(filt_ds_path) if os.path.isdir(os.path.join(filt_ds_path, d)) and "sub" in d]
    for subject_name in subject_names:
        if subjects_to_include is not None:
            if subject_name not in subjects_to_include:
                continue
        # load paths
        good_epochs_path = os.path.join(filt_ds_path, "derivatives", "mne-bids-pipeline", subject_name, "eeg", f"{subject_name}_task-casinos_proc-clean_epo.fif")
        raw_beh_path = os.path.join(raw_ds_path, subject_name, "beh", f"{subject_name}_task-casinos_beh.tsv")
        raw_casinos_events = os.path.join(raw_ds_path, subject_name, "eeg", f"{subject_name}_task-casinos_events.tsv")
        if any([not os.path.exists(good_epochs_path), not os.path.exists(raw_casinos_events),not os.path.exists(raw_beh_path)]):
            print(f"{subject_name} is missing either good_epochs_file, raw_beh_path or raw_casinos_events!")
            continue

        # load files
        epochs = mne.read_epochs(good_epochs_path, preload=True)
        beh = pd.read_csv(raw_beh_path, sep='\t')
        beh['prev_outcome'] = beh['outcome'].shift(1)
        raw_condition_event_ids = pd.read_csv(raw_casinos_events, sep='\t')
        raw_condition_event_ids["value"] = raw_condition_event_ids["value"].astype(str)

        # pick channel
        epochs = epochs.pick(picks=[selected_channel])
        event_ids = epochs.events

        for condition in epochs.event_id.keys():
            condition_epochs = epochs[condition]
            condition_event_ids = event_ids[event_ids[:, 2] == epochs.event_id[condition]]
            condition_id = CONDITION_MAPPING[condition]
            raw_condition_event_ids["number"] = raw_condition_event_ids["value"].apply(extract_number)
            condition_condition_event_ids = raw_condition_event_ids[raw_condition_event_ids["number"] == condition_id]
            filtered_condition_idxs = [i for i, sample in enumerate(condition_condition_event_ids["sample"]) if int(sample) - 1 in condition_event_ids[:, 0]]
            task, prob, outcome = convert_condition_id_to_beh_names(condition_id)
            filtered_beh = filter_df_by_kwargs(beh, task=task, prob=prob, outcome=outcome)
            filtered_beh.index = range(len(filtered_beh))
            filtered_beh = filtered_beh.loc[filtered_condition_idxs]
            for i, epoch in enumerate(condition_epochs):
                epoch = epoch.flatten() * 1000000
                epoch = epoch[440:541]
                max_voltage = np.max(epoch)
                mean_voltage = np.mean(epoch)
                outcome = "win" if filtered_beh.iloc[i]["outcome"] == 1 else "loss"
                if filtered_beh.iloc[i]["prev_outcome"] == 1:
                    prev_outcome = "win"
                elif filtered_beh.iloc[i]["prev_outcome"] == 0:
                    prev_outcome = "loss"
                elif filtered_beh.iloc[i]["prev_outcome"] == -1:
                    prev_outcome = "invalid"
                else:
                    raise Exception("Something went wrong with the prev_outcome assignment")
                if filtered_beh.iloc[i]["task"] == 1:
                    task = "low"
                elif filtered_beh.iloc[i]["task"] == 2:
                    task = "mid"
                elif filtered_beh.iloc[i]["task"] == 3:
                    task = "high"
                else:
                    raise Exception("Something went wrong with the task assignment")
                cue = "low" if filtered_beh.iloc[i]["prob"] == 50 else "high"
                mean_voltages.append(mean_voltage)
                max_voltages.append(max_voltage)
                cues.append(cue)
                outcomes.append(outcome)
                prev_outcomes.append(prev_outcome)
                subjects.append(subject_name)
                tasks.append(task)
    df = pd.DataFrame({
        "subject": subjects,
        "task": tasks,
        "cue": cues,
        "prev_outcome": prev_outcomes,
        "outcome": outcomes,
        "mean_voltage": mean_voltages,
        "max_voltage": max_voltages
    })
    if output_dir is not None:
        df.to_csv(output_dir, index=False)
    return df

### Start Linear Modeling

Before we start creating a Linear Model, lets first see how our data, i.e. the mean/max voltages values, looks like. 

In [31]:
data = retrieve_data(selected_channel=SELECTED_CHANNEL, output_dir=OUTPUT_DIR, raw_ds_path=RAW_DS_PATH, filt_ds_path=FILT_DS_PATH, subjects_to_include=SUBJECTS_TO_INCLUDE)

# pair up task and cue
data['task_cue'] = data['task'] + '_' + data['cue']
# pair up task_cue with outcome
data['task_cue_prev_outcome'] = data['task_cue'] + '_' + data['prev_outcome']
data["task_cue_prev_outcome_outcome"] = data['task_cue_prev_outcome'] + "_" + data["outcome"]
# Plotting
plt.figure(figsize=(8, 6)) 
plt.boxplot(data[VOLTAGE_TO_ANALYZE])
plt.ylabel(f'{VOLTAGE_TO_ANALYZE}')
plt.title(f'{VOLTAGE_TO_ANALYZE} Values Visualized as a Box Plot')
plt.show()

data['squared_errors'] = (data[VOLTAGE_TO_ANALYZE] - data[VOLTAGE_TO_ANALYZE].mean()) ** 2 
sse = data['squared_errors'].sum()
print("Sum of Squared Errors (SSE):", sse)
data[VOLTAGE_TO_ANALYZE].describe()

Reading /mnt/d/PycharmProjects/EEG2324Brownie/Dataset/ds004147-filtered/derivatives/mne-bids-pipeline/sub-27/eeg/sub-27_task-casinos_proc-clean_epo.fif ...
    Found the data of interest:
        t =    -200.00 ...     600.00 ms
        0 CTF compensation matrices available
Adding metadata with 31 columns
356 matching events found
No baseline correction applied
0 projection items activated


ValueError: zero-size array to reduction operation maximum which has no identity

The calculated SSE is our baseline value regarding Linear Modeling. This is the value we get, when **no** independent variables are considered, i.e. the Linear Model only consists of the intercept which is represented as the mean. As a next step, we want to carefully select predictors for our Linear Model to reduce the SSE and thus understand the baseline variance we currently have better.

## Potential Predictors

Given the paper, 3 predictors can be derived that helps in understanding the variance of the RewP values, namely the Task-Cue Pair, the Outcome and the Performance. Even though we know from the paper, that these 3 predictors are good to be included into our linear modeling, as a sanity check, let us check the relationship to RewP anyway. The variables task and cue are paired together 

#### Task

In [ ]:
analyze_categorical_feature("task_cue", data)

From the box plots and the ANOVA table it is clear that the task variable has a significant effect on the RewP generated. A F-value of 20 indicates that there is large evidence against the null hypothesis (Every Task Group contributes the same to the RewP). As this F-value is also way under the standard significance level of 0.01 we thereby conclude, that the task is a good as a predictor for the linear modeling.

In [ ]:
analyze_categorical_feature("task_cue_prev_outcome_outcome", data)

In [ ]:
# Assuming 'data' is your DataFrame containing the relevant columns
contingency_table = pd.crosstab(data['prev_outcome'], data['outcome'])

# Perform chi-squared test
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("Chi-squared statistic:", chi2)
print("P-value:", p)
print("Degrees of freedom:", dof)
print("Expected frequencies table:")
print(expected)

TODO: write analysis

In [ ]:
analyze_categorical_feature("outcome", data)

#### Performance

In [ ]:
#analyze_numerical_feature("performance", data)

TODO: write analysis

### Self-chosen Predictors

We have decided to look for two more potential predictors, namely the Response Time, i.e. how fast a participant pulls the arm of slot and the Previous Outcome. We hypothesize that both these predictors can be good candidates to understand the variance in the RewP. Let's start with the Response Time. 

###  1. Self-Chosen Predictor: Response Time

We hypothesize that a participant clicks the button faster when confident in their choice. Conversely, when the participant did not learn a pattern yet for a cue or if there is no pattern to learn, i.e. for low cues, the participant might think about which choice to take leading to a longer response time. When participants are confident, i.e. they respond fast, this could influence the RewP generated.


#### Sanity Check for Response Time

In our sanity check we will **only consider the medium task**. In the medium task, where both types of cue are present, we hypothesise that, particularly in this type of casino, it should be rewarding for a participant to be presented with a cue for which the participant has already learned the pattern (represented by the assumed faster click), due to the presence of low cues, and therefore frustrating to learn an assumed pattern for a non-learnable cue. 
We use 3 metrics to check whether the data indicate faster response times on learned trials or not, namely Z-score, percentile of response time error, and absolute response time error.

In [ ]:
subjects_dir = 'Dataset/ds004147-filtered'
subject_dirs = [d for d in os.listdir(subjects_dir) if os.path.isdir(os.path.join(subjects_dir, d)) and "sub" in d]

dfs = []
for subject_dir in subject_dirs:
    df = create_df_from_beh_tsv(os.path.join(subjects_dir, subject_dir, "beh", f"{subject_dir}_task-casinos_beh.tsv"))
    dfs.append(df)
    
df = pd.concat(dfs, ignore_index=True)

# Filter by medium task (task=2)
df2 = df[df['task'] == 2]

# Filter by cue type: Non-Learned (prob=50) and Learned (prob=80, optimal=outcome)
df2_non_learned = df2[(df2['prob'] == 50) | ((df2['prob'] == 80) & (df2['outcome'] == 0))]
df2_learned = df2[(df2['prob'] == 80) & (df2['optimal'] == 1) & (df2['outcome'] == 1)]

# Combine data for box plot
combined_data = pd.concat([df2_non_learned["rt_z_score"], df2_learned["rt_z_score"]], axis=1)
combined_data.columns = ['Non-Learned', 'Learned']

# Box plot for Z-score
plt.figure(figsize=(15, 5))
plt.subplot(1, 3, 1)
plt.boxplot([df2_non_learned["rt_z_score"], df2_learned["rt_z_score"]])
plt.xticks([1, 2], ['Non-Learned', 'Learned'])
plt.ylabel('Response Time Error (Z-score)')
plt.title('Distribution of Response Time Errors (Z-score)')

# Box plot for Percentile
plt.subplot(1, 3, 2)
plt.boxplot([df2_non_learned["rt_percentile"], df2_learned["rt_percentile"]])
plt.xticks([1, 2], ['Non-Learned', 'Learned'])
plt.ylabel('Response Time Percentile')
plt.title('Distribution of Response Time Errors (Percentile)')

# Box plot for Residuals
plt.subplot(1, 3, 3)
plt.boxplot([df2_non_learned["residuals"], df2_learned["residuals"]])
plt.xticks([1, 2], ['Non-Learned', 'Learned'])
plt.ylabel('Error in ms')
plt.title('Distribution of Response Time Errors (absolute)')

plt.tight_layout()
plt.show()

#### Result: No significant differences in response time for learned and non-learned trials


Based on the three box plots showing response time errors for each subject in the medium task, there do not appear to be significant differences between non-learned and learned trials. This suggests that our hypothesis that subjects will pull the lever faster when they are more confident is very likely incorrect.

Several factors may contribute to this result. First, regardless of whether the subject already "knows" which arm to pull, there is an inherent processing time required to determine which action to take. This processing time is influenced by the number of different cues the subject has to remember.

Second, our assumption that confidence leads to faster responses may also be inaccurate. It's worth noting that subjects receive no tangible benefit from faster responses. Therefore, the motivation to respond quickly may not be as important as we originally hypothesized.

**Thus, we conclude that using response time as a predictor is most likely not a good choice for linear modeling, as no difference between learned and unlearned trials can be extracted.**.

### 2. Self-Chosen Predictor: Previous Outcome

For this predictor we hypothesize that the previous outcome might affect the RewP generated on the next trial. For that we are going to create a categorical variable which can take the value 0 for a loss and 1 for a win on the previous outcome. Again, we are first going to plot a box plot to visualize how the previous outcome affects the RewP generated for each task and cue combination, i.e. low-low, mid-low, mid-high and high-high.


In [ ]:
analyze_categorical_feature("prev_outcome", data)

TODO: write analysis

#### After the analysis of all the potential predictors, we will select Task, Cue, Outcome, Performance and Previous Outcome as Predictors for the dependent Variable RewP!


### Remaining Predictors: Task, Cue, Outcome and the Ratio of correct learnable Trial

For the remaining predictors, we don't have to conduct a sanity check, because from the paper it is clear that these predictors are do have a strong relationship with the RewP generated.

### Test For MultiCollinearity
TODO

### Multiple Linear Regression

Numerical independent variables can be represented by just a single variable whereas for a categorical independent variable we have to create n - 1 dummy variables where n is the number of categories. This leads to the following variables:
- Task: categorical; x1 and x2 to represent Low / Mid / High
- Cue: categorical; x3 to represent Low / High
- Outcome: categorical; x4 to represent Win / Loss
- Performance: numerical; x5
- Previous Outcome: categorical; x6 

#### E(RewP) = β₀ + β₁x₁ + β₂x₂ + β₃x₃ + β₄x₄ + β₅x₅ + β₆x₆


In [ ]:
test = pd.get_dummies(data, columns=["task_cue_prev_outcome_outcome"], drop_first=True) 
# model_formula = 'RewP ~ performance + task_cue_low_low + task_cue_mid_high + task_cue_mid_low + outcome_win + prev_outcome_win'
model_formula = f'{VOLTAGE_TO_ANALYZE} ~ task_cue_low_low + task_cue_mid_high + task_cue_mid_low + outcome_win + prev_outcome_win'
model = ols(formula=model_formula, data=test).fit()
print(model.summary())

residuals = model.resid
SSE = np.sum(residuals ** 2)
print("Sum of Squared Errors (SSE):", SSE)

TODO: write Interpretation of results